In [1]:
import copy
from pathlib import Path
import warnings
import random

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

import matplotlib.pyplot as plt

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
# input_length = 168
# output_length = 24

input_length = 336
output_length = 168

show_graph = False

In [3]:
# Group 3
model_paths = [ "Save_File/Group3_1/A.ckpt",
                "Save_File/Group3_1/B.ckpt",
                "Save_File/Group3_1/C.ckpt",
                "Save_File/Group3_1/D.ckpt",
                "Save_File/Group3_1/E.ckpt",
                ]

group_name = ["A", "B", "C", "D", "E"]

group = [
    [34, 40, 42, 41, 4, 10, 11, 12],
    [35, 6, 48, 27, 57, 8, 25, 56, 26, 55, 47, 13, 53, 18, 7, 17, 46],
    [31, 33, 9, 3, 1, 32],
    [29, 38, 43, 58, 15, 22, 39, 54, 23, 44, 45, 37, 52, 2, 14],
    [21, 19, 50, 49, 20, 51, 30, 36, 28, 59, 5, 60, 16, 24]
]

data_path = "./Refined_Data/Grouped_Data/Input_Data2.csv"

In [4]:
dataframes = pd.read_csv(data_path, parse_dates = ["date_time"])

dataframes['num']     =   dataframes['num'].apply(str)
dataframes['day_of_Week']     =   dataframes['day_of_Week'].apply(str)
dataframes['day_of_month']    =   dataframes['day_of_month'].apply(str)
dataframes['24Hour']  =   dataframes['24Hour'].apply(str)
dataframes['holiday'] =   dataframes['holiday'].apply(str)
dataframes['Weekend'] =   dataframes['Weekend'].apply(str)
dataframes['energy_group'] = dataframes['energy_group'].apply(str)
dataframes['hour_cat']=   dataframes['hour_cat'].apply(str)
# dataframes['discomfort_index'] = dataframes['discomfort_index'].apply(str)

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F) + 1e-10))

all_smape = []
part_smape = [[] for i in range(len(group_name))]

In [5]:
random_choice = 3 # random.randrange(3,12)

start_point = -168 * random_choice - 1
duration = 168

In [6]:
for i in range(len(model_paths)):

    print(f"now group : {group_name[i]}")

    for building in sorted(group[i]):
        now_building_dataframe = dataframes.loc[dataframes["num"] == str(building)].copy()
        now_building_dataframe = now_building_dataframe[start_point:start_point+input_length+duration]

        now_best_tft = TemporalFusionTransformer.load_from_checkpoint(model_paths[i])

        original = now_building_dataframe.iloc[0:input_length]["kWH"].tolist()
        prediction = now_building_dataframe.iloc[0:input_length]["kWH"].tolist()  

        total_cycle = ((len(now_building_dataframe) - input_length - output_length) // output_length) + 1

        for k in range(total_cycle):
            now_start_point = k*output_length

            encoder_data = now_building_dataframe.iloc[now_start_point:now_start_point+input_length].copy()
            decoder_data = now_building_dataframe.iloc[now_start_point+input_length:now_start_point+input_length+output_length].copy()
            new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)

            raw_predictions = now_best_tft.predict(new_prediction_data, mode="prediction").numpy().tolist()[0]
            originals = now_building_dataframe.iloc[now_start_point+input_length : now_start_point+input_length+output_length]["kWH"].tolist()

            prediction.extend(raw_predictions)
            original.extend(originals)

            # for next step, change dataframe's original value to predicted value
            for p in range(output_length):
                now_building_dataframe.iloc[now_start_point+input_length+p, now_building_dataframe.columns.get_loc("kWH")] = raw_predictions[p]
            
        if show_graph:
            plt.rcParams["figure.figsize"] = (17,5)
            fig = plt.figure()
            graph = fig.add_subplot(1, 1, 1)
            graph.plot(prediction, color='blue')
            graph.plot(original, color='red')
            plt.show()

        smape_loss = smape(np.array(prediction[168:]), np.array(original[168:]))
        print(f"building : {building}, score : {smape_loss}")
        all_smape.append(smape_loss)
        part_smape[i].append(smape_loss)

        # break
    # break

now group : A
building : 4, score : 10.750062809341822
building : 10, score : 5.729658772495867
building : 11, score : 2.9488910472691994
building : 12, score : 3.2610781121521635
building : 34, score : 8.452672703287698
building : 40, score : 7.348116358478456
building : 41, score : 4.366999327535458
building : 42, score : 6.504465947088189
now group : B
building : 6, score : 2.7443194193980114
building : 7, score : 3.7503283164704184
building : 8, score : 3.4840772625533156
building : 13, score : 1.5269171843040943
building : 17, score : 2.6233336848180153
building : 18, score : 4.6984500101058435
building : 25, score : 5.865577447680843
building : 26, score : 1.4842573956166931
building : 27, score : 4.656374837354462
building : 35, score : 6.136624767403066
building : 46, score : 3.229610712826472
building : 47, score : 1.6376684077947692
building : 48, score : 8.289445331497392
building : 53, score : 1.5164019832519664
building : 55, score : 4.268643078589785
building : 56, score 

In [7]:
print(f"selected time slice : {random_choice}")
print(f"totoal score : {np.mean(np.array(all_smape))}")
for i in range(len(group_name)):
    print(f"{group_name[i]} score ({len(group[i])}) : {np.mean(np.array(part_smape[i]))}")

selected time slice : 3
totoal score : 3.4111913977807418
A score (8) : 6.170243134706106
B score (17) : 3.845110380178353
C score (6) : 0.5844021627509132
D score (15) : 2.75544522359047
E score (14) : 3.221755071128649
